In [1]:
import os
import re
import sys
from pathlib import Path
from collections import defaultdict

def find_linked_files(file_content, current_file_path):
    """
    Parse file content and find all linked local files.
    Returns a list of resolved absolute paths.
    """
    current_file = os.path.abspath(current_file_path)
    current_dir = os.path.dirname(current_file)
    
    # Enhanced regex to handle various import formats
    import_pattern = re.compile(r"""
        (?:import|export)\s+                     # import or export keyword
        (?:(?:[\w*\s{},]*)\s+from\s+)?          # optional from clause
        ['"]([^'"]+)['"]                        # the actual path in quotes
    """, re.VERBOSE)
    
    linked_files = []
    linked_paths = []
    
    for match in import_pattern.findall(file_content):
        import_path = match.strip()
        
        # Skip external modules (React, Next.js, Axios, etc.)
        if not import_path.startswith((".", "/")):
            continue
        
        linked_paths.append(import_path)
        
        if import_path.startswith("/"):
            import_path = "." + import_path
        
        # Resolve path relative to the current file
        abs_path = os.path.normpath(os.path.join(current_dir, import_path))
        # print(abs_path)
        
        # Handle directory imports with index files
        if os.path.isdir(abs_path):
            for index_file in ["index.js", "index.jsx", "index.ts", "index.tsx"]:
                index_path = os.path.join(abs_path, index_file)
                if os.path.exists(index_path):
                    linked_files.append((import_path, index_path))
                    break
            continue
        
        # Check if file exists with or without extension
        if os.path.exists(abs_path):
            linked_files.append((import_path, abs_path))
            continue
            
        # Try adding extensions if the file doesn't exist directly
        valid_extensions = [".js", ".jsx", ".ts", ".tsx"]
        for ext in valid_extensions:
            if os.path.exists(abs_path + ext):
                linked_files.append((import_path, abs_path + ext))
                break
    
    return linked_paths, linked_files

def find_api_endpoints(file_content):
    """
    Extract all API endpoints used in axios calls or fetch requests.
    """
    # Pattern for axios calls like axios.get('/api/...')
    axios_pattern = re.compile(r"""
        axios\.(get|post|put|delete|patch)\s*\(\s*
        (?:
            [`'"]([^`'"]+)[`'"]|  # Standard string
            `([^`]+)`             # Template string without variables
        )
    """, re.VERBOSE)
    
    # Pattern for template strings with variables like `/api/user/${id}`
    template_pattern = re.compile(r"""
        axios\.(get|post|put|delete|patch)\s*\(\s*
        `([^`]+)`                 # Template string with variables
    """, re.VERBOSE)
    
    endpoints = []
    
    # Find standard axios calls
    for match in axios_pattern.findall(file_content):
        method, endpoint1, endpoint2 = match
        endpoint = endpoint1 or endpoint2
        if endpoint and endpoint.startswith('/api/'):
            endpoints.append((method, endpoint))
    
    # Find template string endpoints and extract variables
    for match in template_pattern.findall(file_content):
        method, template = match
        if '${' in template and template.startswith('/api/'):
            # Extract template variables
            var_pattern = re.compile(r'\${([^}]+)}')
            variables = var_pattern.findall(template)
            
            endpoints.append((method, template, variables))
    
    return endpoints

def load_file_content(file_path):
    """Load file content from specified path"""
    try:
        with open(file_path, "r", encoding="utf-8") as f:
            return f.read()
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return None

def fetch_linked_files_content(file_path):
    """Fetch content of the main file and all its linked files"""
    content = load_file_content(file_path)
    if not content:
        return {}
    
    # Find linked files
    linked_paths, linked_files = find_linked_files(content, file_path)
    
    # Find API endpoints
    api_endpoints = find_api_endpoints(content)
    
    # Create a dictionary with original file content
    result = {
        file_path: {
            "content": content,
            "import_paths": linked_paths,
            "api_endpoints": api_endpoints
        }
    }
    
    # Add content of linked files
    for import_path, abs_path in linked_files:
        linked_content = load_file_content(abs_path)
        if linked_content:
            # Recursively analyze each linked file as well
            linked_api_endpoints = find_api_endpoints(linked_content)
            result[abs_path] = {
                "content": linked_content,
                "import_path": import_path,
                "api_endpoints": linked_api_endpoints
            }
    
    return result

def analyze_api_structure(files_data):
    """
    Analyze the API structure based on endpoints used in the files.
    """
    print(files_data)
    api_structure = defaultdict(list)
    
    # Group endpoints by base path
    for file_path, data in files_data.items():
        if "api_endpoints" not in data:
            continue
            
        for endpoint_info in data["api_endpoints"]:
            if len(endpoint_info) == 2:  # Standard endpoint
                method, endpoint = endpoint_info
                parts = endpoint.strip('/').split('/')
                if len(parts) > 1:
                    base_path = f"/{parts[0]}/{parts[1]}"
                    api_structure[base_path].append({
                        "method": method,
                        "endpoint": endpoint,
                        "file": file_path,
                        "dynamic": False
                    })
            else:  # Template string with variables
                method, endpoint, variables = endpoint_info
                parts = endpoint.strip('/').split('/')
                if len(parts) > 1:
                    base_path = f"/{parts[0]}/{parts[1]}"
                    api_structure[base_path].append({
                        "method": method,
                        "endpoint": endpoint,
                        "variables": variables,
                        "file": file_path,
                        "dynamic": True
                    })
    
    return api_structure

def main():
    # Get file path from command line or use default
    if len(sys.argv) > 1:
        file_path = sys.argv[1]
    else:
        # Using the file from the example
        file_path = "test.js"
    
    # Ensure the file exists
    if not os.path.exists(file_path):
        print(f"Error: File {file_path} not found")
        return
    
    # Fetch all linked files and their content
    files_data = fetch_linked_files_content(file_path)
    
    # Analyze API structure
    api_structure = analyze_api_structure(files_data)
    
    # Print results
    print(f"\n🔍 Analysis of {file_path}:\n")
    
    # Print main file imports
    main_imports = files_data[file_path]['import_paths']
    print(f"📁 Found {len(main_imports)} local imports:")
    for path in main_imports:
        print(f"  ↪ {path}")
    
    # Print linked files content
    linked_files = [path for path in files_data if path != file_path]
    print(f"\n📂 Resolved {len(linked_files)} linked files:")
    for abs_path in linked_files:
        data = files_data[abs_path]
        print(f"\n✅ {data['import_path']} → {abs_path}")
        print(f"   File size: {len(data['content'])} characters")
        # Print a small preview of the file content
        preview = data['content'][:100].replace('\n', ' ').strip()
        if len(data['content']) > 100:
            preview += "..."
        print(f"   Preview: {preview}")
    
    # Print API endpoints
    all_endpoints = sum(len(data.get("api_endpoints", [])) for data in files_data.values())
    print(f"\n🌐 Found {all_endpoints} API endpoints across all files:")
    
    for base_path, endpoints in api_structure.items():
        print(f"\n  API Group: {base_path}")
        for endpoint_info in endpoints:
            method = endpoint_info["method"].upper()
            endpoint = endpoint_info["endpoint"]
            file = os.path.basename(endpoint_info["file"])
            
            if endpoint_info["dynamic"]:
                variables = ", ".join(endpoint_info["variables"])
                print(f"    {method} {endpoint} (dynamic, vars: {variables}) - in {file}")
            else:
                print(f"    {method} {endpoint} - in {file}")
    
    # Print missing files
    missing_files = [
        path for path in main_imports 
        if not any(data.get('import_path') == path for _, data in files_data.items() if _ != file_path)
    ]
    
    if missing_files:
        print(f"\n⚠️ Could not resolve {len(missing_files)} files:")
        for path in missing_files:
            print(f"  ❌ {path}")

if __name__ == "__main__":
    main()

Error: File --f=c:\Users\Apurav\AppData\Roaming\jupyter\runtime\kernel-v3547266b36a9688219f9d355cf4979bfd3a3c655e.json not found
